Potrzebne funkcje:

In [2]:
using QI
star(A, B) = sqrtm(B)*A*sqrtm(B) # operator z (#2)
# kron - iloczyn Kroneckera
maxmix(d) = eye(d)/d
id(d) = eye(d)

id (generic function with 1 method)

### Prawdopodobieństwa $\rho_A$, $\rho_B$ oraz warunkowe $\rho_{C | AB}$

Są one zdefiniowane tylko i wyłącznie przez warunki zadania:

* Zdarzenie $A$ - numer drzwi, w których znajduje się nagroda
* Zdarzenie $B$ - numer drzwi, które wybrał gracz
* Zdarzenie $C$ - numer drzwi, które otwarł prowadzący

Prawdopodobieństwa zdarzeń $A$ i $B$ mają rozkład jednostajny. W przypadku klasycznym są one niezależne.

In [3]:
roA=diagm([1/3, 1/3, 1/3])
roB=diagm([1/3, 1/3, 1/3])

roCwAB = diagm([
        0,1/2,1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        0,1,0, #A=0, B=2
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); # rozkład warunkowy C|AB (#12)

### Rozkład AB klasyczny  - A i B są zdarzeniami niezależnymi

Macierz go opisujacą można uzyskać dwoma sposobami - używając iloczynu Kroneckera między rozkładami A i B, lub zdefiniować splątanie kwantowe między A oraz B, które zachowuje się tak, jakby go nie było, bo każda kombinacja stanów AB jest równo prawdopodobna.

In [4]:
AB_kron = kron(roA, roB)

AB_ketbra = 1/9 * (
    ket(0,9)*bra(0,9) + 
    ket(1,9)*bra(1,9) + 
    ket(2,9)*bra(2,9) + 
    ket(3,9)*bra(3,9) + 
    ket(4,9)*bra(4,9) + 
    ket(5,9)*bra(5,9) +
    ket(6,9)*bra(6,9) + 
    ket(7,9)*bra(7,9) + 
    ket(8,9)*bra(8,9)
)

print(real(AB_kron) == real(AB_ketbra))
AB_classic = AB_kron

true

9×9 Array{Float64,2}:
 0.111111  0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.111111  0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.111111  0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.111111     0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.111111  0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.111111  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.111111

### Rozkład AB splątany:

$AB_{entangled} = \frac{1}{3} ( | 00 \rangle + |11 \rangle + |22 \rangle)(\langle 00 | + \langle 11 | + \langle 22 |) $

In [5]:
AB_entangled=1/3*(ket(0,9)+ket(4,9)+ket(8,9))* (bra(0,9)+bra(4,9)+bra(8,9)) 
trace(AB_entangled)

1.0 + 0.0im

### Rozkład AB klasyczny lub splątany

Tutaj wybieramy, czy dalej omawiane eksperymenty będą na układzie klasycznym czy splątanym:

In [6]:
AB = AB_classic #układ klasyczny
# AB = AB_entangled #układ splątany

9×9 Array{Float64,2}:
 0.111111  0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.111111  0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.111111  0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.111111     0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.111111  0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.111111  0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.111111

### Rozkład ABC

In [7]:
# wyliczenie roABC znając rozkład AB
roABC = star(roCwAB, kron(AB, id(3))) #equation between (#19 and #20)

trace(roABC)

1.0000000000000002

## Rozkład A bez znajomości C i B

W obu przypadkch powinien być jednostajny - $[ \frac{1}{3}, \frac{1}{3},\frac{1}{3}] $

In [8]:
roA = ptrace(roABC, [3,3,3], [2,3])
real(roA)

3×3 Array{Float64,2}:
 0.333333  0.0       0.0     
 0.0       0.333333  0.0     
 0.0       0.0       0.333333

W poniższych przykładach liczymy rozkłady A przy znanym C, B lub obu. Korzystamy z transformacji analogicznych do np.:

$$
\rho_{ABC} \rightarrow \rho_{A,B=b,C}
$$

Opisanej w równaniu #14

## Rozkład A przy znanym C

Wnioskowanie gdzie jest nagroda na podstawie drzwi, które otwarł prowadzący.
Jeżeli wiemy, które drzwi otwarł prowadzący, to wiemy że nagroda znajduje się w jednych z pozostałych dwojga drzwi. 

Czyli **wydaje nam się, że** dwoje drzwi o numerach $\neq C$ powinny mieć prawdopodobieństwa $\frac{1}{2}$ niezależnie od tego, czy między $A$ a $B$ są efekty kwantowe - a drzwi o numerze $=C$ - prawdopodobieństwo 0

In [9]:
c = 0
coper = ketbra(c,c,3)

# zdarzenie C=1 w systemie
roABc = (kron(kron(id(3), id(3)), coper) * roABC * kron(kron(id(3), id(3)), coper) ) / 
trace(kron(kron(id(3), id(3)), coper)  * roABC); 

roA = ptrace(roABc, [3,3,3], [2,3])
real(roA)

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.5

Okazuje się jednak nieintuicyjnie, że gdy nieokreślone jest $B$, to nieokreślone pozostaje $A$ - nawet mimo znajomości $C$.

## Rozkład A przy znanym B

W przypadku klasycznym powininen być jednostajny - $[ \frac{1}{3}, \frac{1}{3},\frac{1}{3}] $, jako że są to zdarzenia niezależne.

W przypadku splątanym, $A=B$ powinno mieć prawdopodobieństwo = 1 

In [10]:
b = 0
boper = ketbra(b,b,3) #roB

# zdarzenie b=0 w systemie
roAbC = (kron(kron(id(3), boper), id(3)) * roABC * kron(kron(id(3), boper), id(3))) / 
trace(kron(kron(id(3), boper), id(3)) * roABC); 

roA = ptrace(roAbC, [3,3,3], [2,3])
real(roA)

3×3 Array{Float64,2}:
 0.333333  0.0       0.0     
 0.0       0.333333  0.0     
 0.0       0.0       0.333333

## Rozkład A przy znanych B oraz C

W przypadku klasycznym, prawdopodobieństwo $A=B$ wynosi $\frac{1}{3}$, $A=C$ wynosi 0, zaś $A\neq B \neq C $ wynosi $\frac{2}{3}$

W przypadku splątanym, $A=B$ powinno mieć prawdopodobieństwo = 1 

In [55]:
b = 0
boper = ketbra(b,b,3)
boper= boper
c = 1
coper = ketbra(c,c,3)

# zdarzenie b=0 w systemie
roAbC = (kron(kron(id(3), boper), id(3)) * roABC * kron(kron(id(3), boper), id(3))) / 
trace(kron(kron(id(3), boper), id(3)) * roABC); 

#zdarzenie c=1 w systemie
roAbc = (kron(kron(id(3), id(3)), coper) * roAbC * kron(kron(id(3), id(3)), coper) ) / 
trace(kron(kron(id(3), id(3)), coper)  * roAbC); 

roA = ptrace(roAbc, [3,3,3], [2,3])
real(roA)

3×3 Array{Float64,2}:
 0.333333  0.0  0.0     
 0.0       0.0  0.0     
 0.0       0.0  0.666667

In [71]:
roAB = AB_entangled
bmat = foldl(kron, 1, [(id(3)), boper])


roAb = (bmat * roAB * bmat) / trace(bmat * roAB)

roA = ptrace(roAB, [3,3], [2])
roB = ptrace(roAB, [3,3], [1])

roAbC = star(roCwAB, kron(kron(roA, roB), id(3)))

# roAbC = (bmat * roABC * bmat ) / 
# trace(bmat * roABC); 

roA = ptrace(roAbC, [3,3,3], [2,3])
real(roA)

3×3 Array{Float64,2}:
 0.333333  0.0       0.0     
 0.0       0.333333  0.0     
 0.0       0.0       0.333333

In [50]:
roAbc = (kron(kron(id(3), boper), coper) * roABC * kron(kron(id(3), boper), coper) ) / 
trace(kron(kron(id(3), boper), coper)  * roABC); 

roA = ptrace(roAbc, [3,3,3], [2,3])
real(roA)

3×3 Array{Float64,2}:
 0.111111  0.0  0.0     
 0.0       0.0  0.0     
 0.0       0.0  0.222222

### C przy znajomości A i B

In [14]:
a = 0
aoper = ketbra(a,a,3) 
b = 0
boper = ketbra(b,b,3)

# zdarzenie b=0 w systemie
roAbC = (kron(kron(id(3), boper), id(3)) * roABC * kron(kron(id(3), boper), id(3))) / 
trace(kron(kron(id(3), boper), id(3)) * roABC); 

#zdarzenie c=1 w systemie
roabC = (kron(aoper, kron(id(3), id(3))) * roAbC * kron(aoper, kron(id(3), id(3))) ) / 
trace(kron(aoper, kron(id(3), id(3)))  * roAbC); 

roC = ptrace(roabC, [3,3,3], [1,2])
real(roC)

3×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.5  0.0
 0.0  0.0  0.5